In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter
import matplotlib
matplotlib.use('TkAgg')

In [4]:

# world height
WORLD_HEIGHT = 7

# world width
WORLD_WIDTH = 10

# wind strength for each column
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
WIND_HOR = [0, 0, 1, 0, 1, 0, 1]

# possible actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

# probability for exploration
EPSILON = 0.1

# Sarsa step size
ALPHA = 0.5

# gamma for Q-Learning and Expected Sarsa
GAMMA = 1

# reward for each step
REWARD = -1.0

START = [3, 0]
GOAL = [3, 7]
ACTIONS = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]

In [5]:
def step(state, action):
    i, j = state
    if action == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), min(j+WIND_HOR[i], WORLD_WIDTH-1)]
    elif action == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), min(j+WIND_HOR[i], WORLD_WIDTH-1)]
    elif action == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1 + WIND_HOR[i], 0)]
    elif action == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1 + WIND_HOR[i] , WORLD_WIDTH - 1)]
    else:
        assert False


In [6]:
# play for an episode
def Sarsa_episode(q_value):
    # track the total time steps in this episode
    time = 0

    # initialize state
    state = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
        action = np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

    # keep going until get to the goal state
    while state != GOAL:
        next_state = step(state, action)
        if np.random.binomial(1, EPSILON) == 1:
            next_action = np.random.choice(ACTIONS)
        else:
            values_ = q_value[next_state[0], next_state[1], :]
            next_action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

        # Sarsa update
        q_value[state[0], state[1], action] += \
            ALPHA * (REWARD + q_value[next_state[0], next_state[1], next_action] -
                     q_value[state[0], state[1], action])
        state = next_state
        action = next_action
        time += 1
    return time

In [7]:
# play for an episode
def Q_learning_episode(q_value):
    # track the total time steps in this episode
    time = 0

    # initialize state
    state = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
        action = np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

    # keep going until get to the goal state
    while state != GOAL:
        next_state = step(state, action)
        if np.random.binomial(1, EPSILON) == 1:
            next_action = np.random.choice(ACTIONS)
        else:
            values_ = q_value[next_state[0], next_state[1], :]
            next_action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

        # Sarsa update
        q_value[state[0], state[1], action] += \
            ALPHA * (REWARD + GAMMA * np.max(q_value[next_state[0], next_state[1], :]) -
                     q_value[state[0], state[1], action])
        state = next_state
        action = next_action
        time += 1
    return time

In [8]:
def figure_6_3():
    q_value1 = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    q_value2 = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    episode_limit = 500

    Sarsa_steps = []
    Q_learning_steps = []
    ep = 0
    while ep < episode_limit:
        Sarsa_steps.append(Sarsa_episode(q_value1))
        Q_learning_steps.append(Q_learning_episode(q_value2))
        # time = episode(q_value)
        # episodes.extend([ep] * time)
        ep += 1

    steps1 = np.add.accumulate(Sarsa_steps)
    steps2 = np.add.accumulate(Q_learning_steps)

    _, ax = plt.subplots()
    ax.plot(steps1, np.arange(1, len(steps1) + 1))
    ax.plot(steps2, np.arange(1, len(steps2) + 1))
    ax.set(xlabel='steps', ylabel='Episodes',
       title='Sarsa vs Q_learning')
    ax.grid()
    plt.savefig('test2.png')
    plt.show()

    # display the optimal policy
    
    Sarsa_optimal_policy = []
    Q_learning_optimal_policy = []
    for i in range(0, WORLD_HEIGHT):
        Sarsa_optimal_policy.append([])
        for j in range(0, WORLD_WIDTH):
            if [i, j] == GOAL:
                Sarsa_optimal_policy[-1].append('G')
                continue
            bestAction = np.argmax(q_value1[i, j, :])
            if bestAction == ACTION_UP:
                Sarsa_optimal_policy[-1].append('U')
            elif bestAction == ACTION_DOWN:
                Sarsa_optimal_policy[-1].append('D')
            elif bestAction == ACTION_LEFT:
                Sarsa_optimal_policy[-1].append('L')
            elif bestAction == ACTION_RIGHT:
                Sarsa_optimal_policy[-1].append('R')
    print('Sarsa_optimal_policy is:')
    for row in Sarsa_optimal_policy:
        print(row)
        
    for i in range(0, WORLD_HEIGHT):
        Q_learning_optimal_policy.append([])
        for j in range(0, WORLD_WIDTH):
            if [i, j] == GOAL:
                Q_learning_optimal_policy[-1].append('G')
                continue
            bestAction = np.argmax(q_value2[i, j, :])
            if bestAction == ACTION_UP:
                Q_learning_optimal_policy[-1].append('U')
            elif bestAction == ACTION_DOWN:
                Q_learning_optimal_policy[-1].append('D')
            elif bestAction == ACTION_LEFT:
                Q_learning_optimal_policy[-1].append('L')
            elif bestAction == ACTION_RIGHT:
                Q_learning_optimal_policy[-1].append('R')
    print('Sarsa_optimal_policy is:')
    for row in Q_learning_optimal_policy:
        print(row)
    print('Wind strength for each column:\n{}'.format([str(w) for w in WIND]))
    print('Wind strength for each row:\n{}'.format([str(w) for w in WIND_HOR]))

In [9]:
figure_6_3()

Sarsa_optimal_policy is:
['D', 'D', 'U', 'U', 'L', 'L', 'L', 'L', 'L', 'L']
['D', 'D', 'L', 'L', 'L', 'L', 'D', 'U', 'L', 'U']
['D', 'D', 'L', 'U', 'D', 'L', 'D', 'L', 'D', 'L']
['D', 'D', 'L', 'L', 'L', 'L', 'L', 'G', 'U', 'D']
['R', 'D', 'D', 'D', 'D', 'R', 'D', 'D', 'L', 'D']
['D', 'L', 'R', 'R', 'R', 'R', 'R', 'L', 'D', 'D']
['D', 'R', 'R', 'L', 'D', 'L', 'U', 'U', 'U', 'D']
Sarsa_optimal_policy is:
['L', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'D', 'D']
['U', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'D']
['D', 'D', 'R', 'U', 'R', 'L', 'L', 'L', 'D', 'L']
['D', 'R', 'D', 'U', 'L', 'D', 'R', 'G', 'U', 'D']
['R', 'R', 'R', 'D', 'D', 'R', 'D', 'R', 'U', 'D']
['L', 'D', 'D', 'R', 'D', 'R', 'R', 'U', 'D', 'L']
['R', 'R', 'R', 'R', 'D', 'D', 'R', 'U', 'U', 'R']
Wind strength for each column:
['0', '0', '0', '1', '1', '1', '2', '2', '1', '0']
Wind strength for each row:
['0', '0', '1', '0', '1', '0', '1']
